In [1]:
from net.tokenizer import SubWordEmbReaderUtil
from utils.load import MetaLoader, TrainLoader

In [3]:
swer_path = './sstm_v0p5_deploy/sstm_v4p49_np_n36134_d128.dat'
swer = SubWordEmbReaderUtil(swer_path)

meta_path = '../../datasets/FH_2024/subtask3/mdata.wst.txt.2023.08.23'
meta_loader = MetaLoader(path=meta_path, swer=swer)
img2id, id2img, img_similarity = meta_loader.get_dataset()


<Initialize subword embedding>
loading= ./sstm_v0p5_deploy/sstm_v4p49_np_n36134_d128.dat


In [2]:
train_path = "../../datasets/FH_2024/subtask3/task1.ddata.wst.txt"
train_loader = TrainLoader(path=train_path)
train_dataset = train_loader.get_dataset()

In [4]:
print(train_dataset[0])

{'description': ['안녕_하 세 요 코디 봇 입 니다 무엇 을 도와 드릴_까 요', '최근 에 열린 꽃 축제 에 가_려고 하_는데 그때 입 을 스커트 를 포함_한 의상 추천_해 주 세 요', '원하 시 는 스커트 기장 이 있 으신가 요', '중간 기장 으로 보여_주 세 요', '겉옷 이 포함_된 코디로 추천_해 드릴_까 요', '얇 은 가디건 으로 추천 부탁 드려_요', '네 반영 하_여 추천 드리 겠 습니다 잠시 만 기다려_주 세 요', '아이보리 색상 의 머메이드 형 스커트 와 부드러운 소재 의 베이지 색상 가디건 을 포함_한 코디 를 추천_해 드립 니다 마음 에 드 시_나 요', '상의 와 신발 은 캐쥬얼 한 디자인 이 마음 에 들_어 요 그런데 가디건 은 길이 가 조금 긴 것 같_아 짧 은 의상 으로 치마 는 활동_하 기 편한 스커트 로 부탁 드려_요', '네 가디건 은 원하 시 는 색상 이 있_나 요', '베이지색 계열 로 보여_주 세 요', '네 반영 하_여 다시 추천 드리 겠 습니다 잠시 만 기다려_주 세 요', '퍼프 소매 디자인 이 가미 된 베이지 색상 의 가디건 과 넛 넛 한 핏 으로 활동_하 기 편한 플레어 형 스커트 를 추천_해 드립 니다 마음 에 드 시_나 요', '가디건 은 핏 이 마음 에 드_는데 스커트 는 때 가 탈 것 같_아 다른 색상 으로 보여_주 세 요', '네 어두운 색상 으로 다시 추천_해 드리 겠 습니다 잠시 만 기다려_주 세 요', '종아리 까지 오 는 블랙 색상 의 스커트 입 니다 마음 에 드 시_나 요', '색상 이랑 디자인 이 튀 지_않 고 움직이 기 편해 보여 마음 에 들_어 요', '마음 에 드 셨_다 니 다행 입 니다', '선택_하 신 아이템 으로 구성_된 최종 코디 입니다 마음 에 드 시_나 요', '네 스타일 에 신경_쓰 면서_도 간단_하 게 입_고 갈_수 있 을 것 같_아 마음 에 들_어 요', '마음 에 드 셨_다 니 다행 입 니다', '코디 봇 을 이용_해 주 셔 서 감사_합 니다'], 